In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [ ]:
stopwords=nltk.corpus.stopwords.words('english')
ps=nltk.PorterStemmer()

In [ ]:
from google.colab import files
uploaded=files.upload()

Saving SMSSpamCollection.tsv to SMSSpamCollection.tsv


In [ ]:
data=pd.read_csv("SMSSpamCollection.tsv",sep='\t')
data.columns=['label','body_text']
data.head(5)

,label,body_text
0,spam,Free entry in 2 a wkly comp to win FA Cup fina...
1,ham,"Nah I don't think he goes to usf, he lives aro..."
2,ham,Even my brother is not like to speak with me. ...
3,ham,I HAVE A DATE ON SUNDAY WITH WILL!!
4,ham,As per your request 'Melle Melle (Oru Minnamin...


In [ ]:
def count_punct(text):
  count=sum([1 for char in text if char in string.punctuation])
  return round(count/(len(text)-text.count(" ")),3)*100;

def clean_text(text):
  text="".join([char.lower() for char in text if text not in string.punctuation])
  tokens=re.split('\W+',text)
  tokens=[ps.stem(word) for word in text if word not in stopwords]
  return tokens

data['body_len']=data['body_text'].apply(lambda x:len(x)-x.count(" "))
data['punct%']=data['body_text'].apply(lambda x:count_punct(x))


tfidf_vect=TfidfVectorizer(analyzer=clean_text)
X_tfidf=tfidf_vect.fit_transform(data['body_text'])

X_features=pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis=1)
X_features.head(5)

,body_len,punct%,0,1,2,3,4,5,6,7,...,72,73,74,75,76,77,78,79,80,81
0,128,4.7,0.0,0.0,0.576515,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.677140,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.710562,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.661315,0.523813,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.566179,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


****RandomForest Classifier****


In [ ]:
from sklearn.ensemble import RandomForestClassifier
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_check_feature_names', '_check_n_features', '_compute_oob_predictions', '_estimator_type', '_get_oob_predictions', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_parameter_constraints', '_repr_html_', '_repr_html_inner', '_repr_mimebundle_', '_required_parameters', '_set_oob_score_and_attributes', '_validate_X_predict', '_validate_data', '_validate_estimator', '_validate_params', '_validate_y_class_weight', 'apply', 'base_estimator_', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predic

***Random Forest using Cross validation***

In [ ]:
from sklearn.model_selection import KFold,cross_val_score

In [ ]:
rf=RandomForestClassifier(n_jobs=-1)
k_fold=KFold(n_splits=5)
X_features.columns = X_features.columns.astype(str)
cross_val_score(rf,X_features,data['label'],cv=k_fold,scoring="accuracy", n_jobs=-1)

array([0.98025135, 0.98384201, 0.98113208, 0.97843666, 0.98382749])

***Random Forest Classifier using HoldOut***

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_features,data['label'],test_size=0.2)

from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,max_depth=20, n_jobs=-1)
rf_model=rf.fit(X_train,y_train)


In [ ]:
sorted(zip(rf_model.feature_importances_,X_train.columns),reverse=True)

[(0.16619708305556308, '18'),
 (0.12609058190807662, '26'),
 (0.10973855099375214, '19'),
 (0.07044715074257503, '25'),
 (0.05932605979580464, '21'),
 (0.05547828668943569, '23'),
 (0.03908621077319388, '24'),
 (0.03620655802008461, 'body_len'),
 (0.030757119643121552, '20'),
 (0.027349507610131284, '27'),
 (0.02660993836536293, '66'),
 (0.02229674551064106, '2'),
 (0.019216029297814526, '45'),
 (0.018961973332101437, '22'),
 (0.015848296057106876, '17'),
 (0.015314383001973006, '56'),
 (0.014523361340788703, '41'),
 (0.011607385495343412, '50'),
 (0.011256496282881287, '16'),
 (0.009477569576084179, '3'),
 (0.008503291408348674, '52'),
 (0.008370585243184932, '42'),
 (0.007597674259987629, 'punct%'),
 (0.006669825766969796, '49'),
 (0.006094129487823832, '44'),
 (0.0060228639321372075, '55'),
 (0.00594458275320056, '48'),
 (0.00557587084864929, '53'),
 (0.005422190325264567, '43'),
 (0.005417688880675446, '40'),
 (0.005042209501840207, '54'),
 (0.0045412531662153135, '32'),
 (0.004356

In [ ]:
y_pred=rf_model.predict(X_test)
precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')
print("precision={}, recall={}, fscore={}, support={}".format(precision,recall,fscore,support))


precision=0.9772727272727273, recall=0.8657718120805369, fscore=0.9181494661921707, support=None


***Random Forest Classifier using Grid Search***

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

In [ ]:
X_features.head(5)

,body_len,punct%,0,1,2,3,4,5,6,7,...,72,73,74,75,76,77,78,79,80,81
0,128,4.7,0.0,0.0,0.576515,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.677140,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.710562,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.661315,0.523813,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.566179,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_features,data['label'],test_size=0.2)

In [ ]:
def train_rf(n_est,depth):
  rf=RandomForestClassifier(n_estimators=n_est,max_depth=depth,n_jobs=-1)
  rf_model=rf.fit(X_train,y_train)
  y_pred=rf_model.predict(X_test)
  precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')
  print("Est:{} / Depth:{} ---- Precision:{} / Recall:{} / Accuracy:{}".format(n_est,depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))


In [ ]:
for n_est in [10,50,100]:
  for depth in [10,20,30,None]:
    train_rf(n_est,depth)

Est:10 / Depth:10 ---- Precision:0.969 / Recall:0.862 / Accuracy:0.978
Est:10 / Depth:20 ---- Precision:0.984 / Recall:0.855 / Accuracy:0.979
Est:10 / Depth:30 ---- Precision:0.984 / Recall:0.855 / Accuracy:0.979
Est:10 / Depth:None ---- Precision:0.977 / Recall:0.869 / Accuracy:0.98
Est:50 / Depth:10 ---- Precision:0.984 / Recall:0.869 / Accuracy:0.981
Est:50 / Depth:20 ---- Precision:0.985 / Recall:0.883 / Accuracy:0.983
Est:50 / Depth:30 ---- Precision:0.977 / Recall:0.862 / Accuracy:0.979
Est:50 / Depth:None ---- Precision:0.984 / Recall:0.869 / Accuracy:0.981
Est:100 / Depth:10 ---- Precision:0.984 / Recall:0.869 / Accuracy:0.981
Est:100 / Depth:20 ---- Precision:0.977 / Recall:0.869 / Accuracy:0.98
Est:100 / Depth:30 ---- Precision:0.984 / Recall:0.876 / Accuracy:0.982
Est:100 / Depth:None ---- Precision:0.984 / Recall:0.869 / Accuracy:0.981


****Evaluating Machine Learning With GridSearchCV****


In [ ]:
#T-IDF
tfidf_vect=TfidfVectorizer(analyzer=clean_text);
X_tfidf=tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat=pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis=1);


In [ ]:
#COUNT-VECTORIZER
count_vect=CountVectorizer(analyzer=clean_text)
X_count=count_vect.fit_transform(data['body_text'])
X_count_feat=pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_count.toarray())],axis=1)


****Exploring parameter using GridSearchCV****

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
rf=RandomForestClassifier()
param={'n_estimators':[10,150,300],
       'max_depth':[30,60,90,None]}
gs=GridSearchCV(rf,param,cv=5,n_jobs=-1)
X_tfidf_feat.columns=X_tfidf_feat.columns.astype(str)
gs_fit=gs.fit(X_tfidf_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
7,4.245056,1.007478,0.147296,0.048801,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.982047,0.984740,0.982031,0.980234,0.982929,0.982396,0.001463,1
2,5.329233,1.642022,0.113589,0.051954,30,300,"{'max_depth': 30, 'n_estimators': 300}",0.982944,0.984740,0.982929,0.979335,0.982031,0.982396,0.001765,2
10,4.530297,1.499420,0.074109,0.037855,None,150,"{'max_depth': None, 'n_estimators': 150}",0.981149,0.985637,0.981132,0.980234,0.982929,0.982216,0.001921,3
11,4.428040,0.261163,0.101003,0.030433,None,300,"{'max_depth': None, 'n_estimators': 300}",0.982944,0.985637,0.980234,0.979335,0.982929,0.982216,0.002235,4
1,2.908777,0.608948,0.067592,0.025482,30,150,"{'max_depth': 30, 'n_estimators': 150}",0.981149,0.985637,0.981132,0.980234,0.980234,0.981677,0.002021,5


In [ ]:
rf=RandomForestClassifier()
param={'n_estimators':[10,150,300],
       'max_depth':[30,60,90,None]}
gs=GridSearchCV(rf,param,cv=5,n_jobs=-1)
X_count_feat.columns=X_count_feat.columns.astype(str)
gs_fit=gs.fit(X_count_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,1.229169,0.020643,0.047298,0.001531,30,150,"{'max_depth': 30, 'n_estimators': 150}",0.981149,0.987433,0.981132,0.978437,0.982929,0.982216,0.002977,1
5,3.651187,0.817385,0.119852,0.037955,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.981149,0.984740,0.982929,0.978437,0.982929,0.982037,0.002128,2
10,1.827524,0.271896,0.062489,0.011384,None,150,"{'max_depth': None, 'n_estimators': 150}",0.982047,0.985637,0.982031,0.977538,0.982929,0.982036,0.002608,3
7,1.216235,0.027381,0.048504,0.005768,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.982944,0.982944,0.981132,0.979335,0.982929,0.981857,0.001442,4
4,2.570517,0.395574,0.101363,0.019310,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.981149,0.985637,0.982929,0.978437,0.981132,0.981857,0.002373,5


****Gradient Decent Exploring****

In [ ]:
data=pd.read_csv('SMSSpamCollection.tsv',sep='\t')
data.columns=['label','body_text']

def count_punct(text):
  count=sum([1 for char in text if char in string.punctuation])
  return round(count/(len(text)-text.count(" ")),3)*100;

def clean_text(text):
  text="".join([char.lower() for char in text if text not in string.punctuation])
  tokens=re.split('\W+',text)
  tokens=[ps.stem(word) for word in text if word not in stopwords]
  return tokens

data['body_len']=data['body_text'].apply(lambda x:len(x)-x.count(" "))
data['punct%']=data['body_text'].apply(lambda x:count_punct(x))


tfidf_vect=TfidfVectorizer(analyzer=clean_text)
X_tfidf=tfidf_vect.fit_transform(data['body_text'])

X_features=pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis=1)
X_features.head(5)

,body_len,punct%,0,1,2,3,4,5,6,7,...,72,73,74,75,76,77,78,79,80,81
0,128,4.7,0.0,0.0,0.576515,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.677140,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.710562,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.661315,0.523813,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.566179,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [ ]:
X_features.columns=X_features.columns.astype(str)
X_train,X_test,y_train,y_test=train_test_split(X_features,data['label'],test_size=0.2)

****Building own GridSeach****

In [ ]:
def train_gb(n_est,depth,lr):
  gb=GradientBoostingClassifier(n_estimators=n_est,max_depth=depth,learning_rate=lr)
  gb_model=gb.fit(X_train,y_train)
  y_pred=gb_model.predict(X_test)
  precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam', average='binary')
  print("Est:{} / Depth:{} ---- Precision:{} / Recall:{} / Accuracy:{}".format(n_est,depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))



In [ ]:
for n_est in [50,100,150]:
  for depth in [3,7,11,15]:
    for lr in [0.01,.1,1]:
      train_gb(n_est,depth,lr)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est:50 / Depth:3 ---- Precision:0.0 / Recall:0.0 / Accuracy:0.873
Est:50 / Depth:3 ---- Precision:0.963 / Recall:0.929 / Accuracy:0.987
Est:50 / Depth:3 ---- Precision:0.921 / Recall:0.915 / Accuracy:0.979


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est:50 / Depth:7 ---- Precision:0.0 / Recall:0.0 / Accuracy:0.873
Est:50 / Depth:7 ---- Precision:0.949 / Recall:0.915 / Accuracy:0.983
Est:50 / Depth:7 ---- Precision:0.891 / Recall:0.929 / Accuracy:0.977


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est:50 / Depth:11 ---- Precision:0.0 / Recall:0.0 / Accuracy:0.873
Est:50 / Depth:11 ---- Precision:0.949 / Recall:0.922 / Accuracy:0.984
Est:50 / Depth:11 ---- Precision:0.928 / Recall:0.915 / Accuracy:0.98


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Est:50 / Depth:15 ---- Precision:0.0 / Recall:0.0 / Accuracy:0.873
Est:50 / Depth:15 ---- Precision:0.942 / Recall:0.915 / Accuracy:0.982
Est:50 / Depth:15 ---- Precision:0.908 / Recall:0.915 / Accuracy:0.978
Est:100 / Depth:3 ---- Precision:0.969 / Recall:0.879 / Accuracy:0.981
Est:100 / Depth:3 ---- Precision:0.964 / Recall:0.943 / Accuracy:0.988
Est:100 / Depth:3 ---- Precision:0.91 / Recall:0.936 / Accuracy:0.98
Est:100 / Depth:7 ---- Precision:0.962 / Recall:0.908 / Accuracy:0.984
Est:100 / Depth:7 ---- Precision:0.942 / Recall:0.915 / Accuracy:0.982
Est:100 / Depth:7 ---- Precision:0.897 / Recall:0.922 / Accuracy:0.977
Est:100 / Depth:11 ---- Precision:0.955 / Recall:0.908 / Accuracy:0.983
Est:100 / Depth:11 ---- Precision:0.942 / Recall:0.915 / Accuracy:0.982
Est:100 / Depth:11 ---- Precision:0.934 / Recall:0.908 / Accuracy:0.98
Est:100 / Depth:15 ---- Precision:0.955 / Recall:0.908 / Accuracy:0.983
Est:100 / Depth:15 ---- Precision:0.935 / Recall:0.922 / Accuracy:0.982
Est:100 

****Gradient Boosting Using GridSearchCV****

In [ ]:
#T-IDF
tfidf_vect=TfidfVectorizer(analyzer=clean_text);
X_tfidf=tfidf_vect.fit_transform(data['body_text'])
X_tfidf_feat=pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_tfidf.toarray())],axis=1);

In [ ]:
#COUNT-VECTORIZER
count_vect=CountVectorizer(analyzer=clean_text)
X_count=count_vect.fit_transform(data['body_text'])
X_count_feat=pd.concat([data['body_len'],data['punct%'],pd.DataFrame(X_count.toarray())],axis=1)


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
gb=GradientBoostingClassifier()
param={'n_estimators':[100,150],
       'max_depth':[7,11,15],'learning_rate':[.1]}
gs=GridSearchCV(gb,param,cv=5,n_jobs=-1)
X_tfidf_feat.columns=X_tfidf_feat.columns.astype(str)
gs_fit=gs.fit(X_tfidf_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,18.087476,1.051191,0.014499,0.000350,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.980251,0.985637,0.983827,0.976640,0.978437,0.980959,0.003336,1
0,17.430926,1.602662,0.029152,0.015748,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.979354,0.985637,0.976640,0.977538,0.979335,0.979701,0.003148,2
2,17.320524,0.611693,0.018816,0.007937,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.979354,0.986535,0.981132,0.973944,0.975741,0.979341,0.004406,3
3,27.414215,1.592020,0.017951,0.000556,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.977558,0.985637,0.982031,0.974843,0.971249,0.978264,0.005098,4
4,22.593874,1.353432,0.019245,0.005665,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.979354,0.983842,0.979335,0.966757,0.974843,0.976826,0.005783,5


In [ ]:
gb=GradientBoostingClassifier()
param={'n_estimators':[100,150],
       'max_depth':[7,11,15],'learning_rate':[.1]}
gs=GridSearchCV(gb,param,cv=5,n_jobs=-1)
X_count_feat.columns=X_count_feat.columns.astype(str)
gs_fit=gs.fit(X_count_feat,data['label'])
pd.DataFrame(gs_fit.cv_results_).sort_values('mean_test_score',ascending=False)[0:5]

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,7.618047,1.011848,0.018566,0.005790,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.981149,0.984740,0.979335,0.976640,0.981132,0.980599,0.002646,1
3,12.508309,1.069132,0.023827,0.008926,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.978456,0.984740,0.978437,0.977538,0.981132,0.980061,0.002631,2
2,7.642073,0.805467,0.020746,0.008246,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.977558,0.983842,0.978437,0.976640,0.979335,0.979162,0.002506,3
0,4.614149,0.869054,0.016218,0.007029,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.979354,0.981149,0.980234,0.975741,0.975741,0.978444,0.002278,4
5,13.574006,1.729415,0.024152,0.008543,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.973968,0.977558,0.974843,0.968553,0.974843,0.973953,0.002958,5


****Model Selection****

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(data[['body_text','body_len','punct%']],data['label'], test_size=.2)


In [ ]:
tfidf_vect=TfidfVectorizer(analyzer=clean_text);
tfidf_vect_fit=tfidf_vect.fit(X_train['body_text'])

tfidf_train=tfidf_vect_fit.transform(X_train['body_text'])
tfidf_test=tfidf_vect_fit.transform(X_test['body_text'])
X_train_vect=pd.concat([X_train[['body_len','punct%']].reset_index(drop=True),pd.DataFrame(tfidf_train.toarray())],axis=1)
X_test_vect=pd.concat([X_test[['body_len','punct%']].reset_index(drop=True),pd.DataFrame(tfidf_test.toarray())],axis=1)

X_train_vect.head(5)


,body_len,punct%,0,1,2,3,4,5,6,7,...,68,69,70,71,72,73,74,75,76,77
0,120,7.5,0.626391,0.082925,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,95,1.1,0.590826,0.000000,0.0,0.0,0.0,0.0,0.101153,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,129,5.4,0.584296,0.129951,0.0,0.0,0.0,0.0,0.000000,0.055982,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,26,7.7,0.698933,0.000000,0.0,0.0,0.0,0.0,0.000000,0.279023,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40,15.0,0.582512,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
import time
X_train_vect.columns=X_train_vect.columns.astype(str)
X_test_vect.columns=X_test_vect.columns.astype(str)
rf=RandomForestClassifier(n_estimators=150,max_depth=None,n_jobs=-1)
start=time.time()
rf_model=rf.fit(X_train_vect,y_train)
y_pred=rf_model.predict(X_test_vect)
end=time.time()

print("fit time={}".format(end-start))
precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam',average='binary')
print("Est:{} / Depth:{} ---- Precision:{} / Recall:{} / Accuracy:{}".format(n_est,depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))




fit time=1.1047255992889404
Est:150 / Depth:15 ---- Precision:0.986 / Recall:0.832 / Accuracy:0.973


In [ ]:
gb=GradientBoostingClassifier(n_estimators=150,max_depth=11,)
start=time.time()
gb_model=gb.fit(X_train_vect,y_train)
y_pred=gb_model.predict(X_test_vect)
end=time.time()

print("fit time={}".format(end-start))
precision,recall,fscore,support=score(y_test,y_pred,pos_label='spam', average='binary')
print("Est:{} / Depth:{} ---- Precision:{} / Recall:{} / Accuracy:{}".format(n_est,depth,round(precision,3),round(recall,3),round((y_pred==y_test).sum()/len(y_pred),3)))


fit time=16.37402081489563
Est:150 / Depth:15 ---- Precision:0.946 / Recall:0.844 / Accuracy:0.969
